In [1]:
# !wget https://github.com/huseinzol05/Malaya-Dataset/raw/master/news/news.zip
# !unzip news.zip

In [2]:
import os

In [3]:
labels = os.listdir('news')
news = ['news/' + i for i in labels if '.json' in i]
labels = [i.replace('.json','') for i in labels]
len(news)

123

In [4]:
import malaya
import re

tokenizer = malaya.preprocessing.SocialTokenizer().tokenize
splitter = malaya.texts._text_functions.split_into_sentences
accept_tokens = ',-.()"\''

def is_number_regex(s):
    if re.match("^\d+?\.\d+?$", s) is None:
        return s.isdigit()
    return True

def detect_money(word):
    if word[:2] == 'rm' and is_number_regex(word[2:]):
        return True
    else:
        return False

def preprocessing(string):
    tokenized = tokenizer(string)
    tokenized = [w.lower() for w in tokenized if len(w) > 1 or w in accept_tokens]
    tokenized = ['<NUM>' if is_number_regex(w) else w for w in tokenized]
    tokenized = ['<MONEY>' if detect_money(w) else w for w in tokenized]
    return tokenized

def clean_label(label):
    string = re.sub('[^A-Za-z\- ]+', ' ', label)
    return re.sub(r'[ ]+', ' ', string.lower()).strip()

In [5]:
labels = [clean_label(label) for label in labels]
rejected_text = ['javascript', 'malaysiakini']

In [6]:
from sklearn.utils import shuffle
import json

maxlen = 150
min_len = 20

x, y = [], []
for no, n in enumerate(news):
    with open(n) as fopen: 
        news_ = json.load(fopen)
    for row in news_:
        if len(row['text'].split()) > min_len:
            split = splitter(row['text'])
            split = [s for s in split if not any([r in s.lower() for r in rejected_text])]
            split = ' '.join(split)
            p = preprocessing(split)
            p = p[:maxlen]
            if len(p) > 10:
                x.append(p)
                y.append(labels[no])

In [7]:
len(x), len(y)

(13963, 13963)

In [8]:
import collections

def build_dataset(words, n_words, atleast=2):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [9]:
import itertools

concat = list(itertools.chain(*x)) + ' '.join(labels).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])
print('filtered vocab size:',len(dictionary))
print("% of vocab used: {}%".format(round(len(dictionary)/vocabulary_size,4)*100))

vocab from size: 61551
Most common words [(',', 89449), ('.', 77833), ('-', 32111), ('yang', 31940), ('the', 29150), ('dan', 28032)]
Sample data [58, 65, 772, 3394, 273, 204, 223, 3534, 2286, 11740] ['kuala', 'lumpur', 'former', 'transport', 'minister', 'tan', 'sri', 'chan', 'kong', 'choy']
filtered vocab size: 37920
% of vocab used: 61.61%


In [10]:
for i in range(len(x)):
    x.append('EOS')

In [11]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [12]:
import tensorflow as tf
from tensor2tensor.utils import beam_search
import numpy as np

In [13]:
def embed_seq(x, vocab_sz, embed_dim, name, zero_pad=True): 
    embedding = tf.get_variable(name, [vocab_sz, embed_dim]) 
    if zero_pad:
        embedding = tf.concat([tf.zeros([1, embed_dim]), embedding[1:, :]], 0) 
    x = tf.nn.embedding_lookup(embedding, x)
    return x

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta


def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x, pad], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          dilation_rate = dilation_rate)
    x = x[:, :-pad_sz, :]
    x = tf.nn.relu(x)
    return x

class Summarization:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 dict_size, learning_rate, 
                 kernel_size = 2, n_attn_heads = 16):

        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])

        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.batch_size = batch_size
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        self.embedding = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        
        self.num_layers = num_layers
        self.kernel_size = kernel_size
        self.size_layer = size_layer
        self.n_attn_heads = n_attn_heads
        self.dict_size = dict_size
        
        self.training_logits = self.forward(self.X, decoder_input)

        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
    def forward(self, x, y, reuse = False):
        with tf.variable_scope('forward',reuse=reuse):
            with tf.variable_scope('forward',reuse=reuse):
                encoder_embedded = tf.nn.embedding_lookup(self.embedding, x)
                decoder_embedded = tf.nn.embedding_lookup(self.embedding, y)
                encoder_embedded += position_encoding(encoder_embedded)

                for i in range(self.num_layers): 
                    dilation_rate = 2 ** i
                    pad_sz = (self.kernel_size - 1) * dilation_rate 
                    with tf.variable_scope('block_%d'%i,reuse=reuse):
                        encoder_embedded += cnn_block(encoder_embedded, dilation_rate, 
                                                      pad_sz, self.size_layer, self.kernel_size)

                g = tf.identity(decoder_embedded)
                for i in range(self.num_layers):
                    dilation_rate = 2 ** i
                    pad_sz = (self.kernel_size - 1) * dilation_rate
                    with tf.variable_scope('decode_%d'%i,reuse=reuse):
                        attn_res = h = cnn_block(decoder_embedded, dilation_rate, 
                                                 pad_sz, self.size_layer, self.kernel_size)
                        C = []
                        for j in range(self.n_attn_heads):
                            h_ = tf.layers.dense(h, self.size_layer//self.n_attn_heads)
                            g_ = tf.layers.dense(g, self.size_layer//self.n_attn_heads)
                            zu_ = tf.layers.dense(encoder_embedded, self.size_layer//self.n_attn_heads)
                            ze_ = tf.layers.dense(encoder_embedded, self.size_layer//self.n_attn_heads)

                            d = tf.layers.dense(h_, self.size_layer//self.n_attn_heads) + g_
                            dz = tf.matmul(d, tf.transpose(zu_, [0, 2, 1]))
                            a = tf.nn.softmax(dz)
                            c_ = tf.matmul(a, ze_)
                            C.append(c_)

                        c = tf.concat(C, 2)
                        h = tf.layers.dense(attn_res + c, self.size_layer)
                        decoder_embedded += h

                return tf.layers.dense(decoder_embedded, self.dict_size)

In [14]:
size_layer = 256
num_layers = 4
embedded_size = 256
learning_rate = 1e-3
batch_size = 32
epoch = 20

In [15]:
def beam_search_decoding(length = 50, beam_width = 5):
    initial_ids = tf.fill([model.batch_size], GO)
    
    def symbols_to_logits(ids):
        x = tf.contrib.seq2seq.tile_batch(model.X, beam_width)
        logits = model.forward(x, ids, reuse = True)
        return logits[:, tf.shape(ids)[1]-1, :]

    final_ids, final_probs, _ = beam_search.beam_search(
        symbols_to_logits,
        initial_ids,
        beam_width,
        length,
        len(dictionary),
        0.0,
        eos_id = EOS)
    
    return final_ids

In [16]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Summarization(size_layer, num_layers, embedded_size, 
                len(dictionary), learning_rate)
model.generate = beam_search_decoding()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [17]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i:
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [18]:
y = [i.split() for i in y]

In [19]:
X = str_idx(y, dictionary)
Y = str_idx(x, dictionary)

In [20]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [21]:
from tqdm import tqdm

for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    pbar = tqdm(
        range(0, len(X), batch_size), desc = 'minibatch loop')
    for k in pbar:
        index = min(k+batch_size, len(X))
        batch_x, seq_x = pad_sentence_batch(X[k: index], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: index], PAD)
        accuracy,loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += accuracy
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    total_loss /= (len(X) / batch_size)
    total_accuracy /= (len(X) / batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

minibatch loop:   0%|          | 1/437 [00:00<01:21,  5.37it/s, accuracy=0.0929, cost=6.6]

epoch: 1, avg loss: 6.923699, avg accuracy: 0.102635


minibatch loop:   0%|          | 1/437 [00:00<01:21,  5.37it/s, accuracy=0.131, cost=6.01]

epoch: 2, avg loss: 5.832101, avg accuracy: 0.154718


minibatch loop:   0%|          | 1/437 [00:00<01:20,  5.41it/s, accuracy=0.12, cost=6.07]

...

epoch: 18, avg loss: 3.174452, avg accuracy: 0.386813


minibatch loop:   0%|          | 1/437 [00:00<01:20,  5.44it/s, accuracy=0.374, cost=3.07]

epoch: 19, avg loss: 3.115246, avg accuracy: 0.394881


minibatch loop: 100%|██████████| 437/437 [01:21<00:00,  5.60it/s, accuracy=0.6, cost=1.89]   

epoch: 20, avg loss: 3.058292, avg accuracy: 0.402294


In [30]:
test_topic = 'isu najib mahathir'

In [31]:
test_topic_idx = str_idx([test_topic.split()], dictionary)
batch_test, _ = pad_sentence_batch(test_topic_idx, PAD)
batch_test

[[29, 134, 112]]

In [32]:
generated = sess.run(model.generate, feed_dict = {model.X: batch_test})[0]
generated.shape

(5, 51)

In [33]:
for g in generated:
    print(' '.join([rev_dictionary[i] for i in g]))

GO - ( ubah saiz teks ) shah alam - percutian perdana menteri , datuk seri najib tun razak dan presiden amerika syarikat ( as ) , prof . datuk kamarudin ahmad razif abdul rahman ( reuters ) - malaysia ' mikhail kolyada of an internal investigation by the end of
GO - ( ubah saiz teks ) shah alam - percutian perdana menteri , datuk seri najib tun razak dan presiden amerika syarikat ( as ) , prof . datuk kamarudin ahmad razif abdul rahman ( reuters ) - malaysia ' mikhail kolyada of an internal investigation on the island ,
GO - ( ubah saiz teks ) shah alam - percutian perdana menteri , datuk seri najib tun razak dan presiden amerika syarikat ( as ) , prof . datuk kamarudin ahmad razif abdul rahman ( reuters ) - malaysia ' mikhail kolyada of an internal investigation on the island of
GO - ( ubah saiz teks ) shah alam - percutian perdana menteri , datuk seri najib tun razak dan presiden amerika syarikat ( as ) , prof . datuk kamarudin ahmad razif abdul rahman ( reuters ) - malaysia ' mikhai